## CALBC vs 8.1 Sketch Model Test

This code tests the CALBC version 8.1 Sketch Model. It utilizes a template file, which is designed to gather test data, and applies this data to the sketch model to generate final results. Test template can be updated [here](https://caltrans.sharepoint.com/:x:/r/sites/DOTHQPMPCalBCUpdate/Shared%20Documents/General/Testbed/Input/testbed_template.xlsm?d=w3bca22e7b5714f5399bb4015c94aa134&csf=1&web=1&e=9AuFn9). To run this code, you'll need to sync the [Testbed folder](https://teams.microsoft.com/l/team/19%3AQrQ3LladNmkiVpFmuCmW2H72CKQz9MrBc2yr1kukBoA1%40thread.tacv2/conversations?groupId=f2d54ae5-490b-4b95-99cf-bafddf8f38b6&tenantId=621b0a64-1740-43cc-8d88-4540d3487556) from [CalB/C Update](https://teams.microsoft.com/l/team/19%3AQrQ3LladNmkiVpFmuCmW2H72CKQz9MrBc2yr1kukBoA1%40thread.tacv2/conversations?groupId=f2d54ae5-490b-4b95-99cf-bafddf8f38b6&tenantId=621b0a64-1740-43cc-8d88-4540d3487556) Teams folder to your local machine. Output will be saved in designated [Output Folder](https://caltrans.sharepoint.com/:f:/r/sites/DOTHQPMPCalBCUpdate/Shared%20Documents/General/Testbed/Output?csf=1&web=1&e=X5VbAM).

In [1]:
pip install xlwings #Installing xlwings

Note: you may need to restart the kernel to use updated packages.


ERROR: Invalid requirement: '#Installing'

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: C:\Users\s159357\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
pip install openpyxl #Installing openpyxl

Note: you may need to restart the kernel to use updated packages.


ERROR: Invalid requirement: '#Installing'

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: C:\Users\s159357\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [3]:
import openpyxl
import xlwings as xw
import pandas as pd
import gcsfs
import time
import shutil
import io
import os
import pandas as pd

In [4]:
user_profile = os.getenv('USERPROFILE') ## Retrieving user profile directory from 'USERPROFILE' environment variable.

In [5]:
# Constructing the base directory path using the user profile directory.
base_dir = os.path.join(
    user_profile, 
    'California Department of Transportation',
    'DOT HQ PMP Cal B C Update - General',
    'Testbed'
)

In [6]:
# Constructing file paths for input, output, and template files.
excel_file_path = os.path.join(base_dir, 'Input', 'cal-bc-8-1-sketch-a11y.xlsm')
output_dir = os.path.join(base_dir, 'Output', 'recalculatedtestresults.xlsm')
template_file_path = os.path.join(base_dir, 'Input', 'testbed_template.xlsm')

In [7]:
# Reading the Excel file as binary and loading it into a workbook object.
with open(excel_file_path, 'rb') as f:
    file_data = f.read()
    
# Loading the workbook from the binary data, preserving any VBA code
    wb = openpyxl.load_workbook(io.BytesIO(file_data), keep_vba=True)

C:\Users\s159357\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\openpyxl\reader\workbook.py:84: UserWarning: File contains an invalid specification for 0. This will be removed
  warn(msg)


In [ ]:
# function to find named ranges, extract sheet name and cell reference and update cell's value with corresponding value from name_value_map
def update_named_ranges(wb, name_value_map):
    for name, new_value in name_value_map.items():
        try:
            # Accessing the defined name ranges in the workbook
            defined_name = wb.defined_names[name]
            
            # Extracting the sheet name and cell reference from the defined name ranges
            sheet_name, cell_reference = defined_name.attr_text.split('!')
            sheet_name = sheet_name.strip("'").strip()
            
            # Accessing the sheet and cell reference
            sheet = wb[sheet_name]
            
            # Getting the current value of the cell
            current_value = sheet[cell_reference].value
            
            # Printing the current value before updating
            print(f"Current value of '{name}' in {sheet_name} ({cell_reference}): {current_value}")
            
            # Updating the value of the cell
            sheet[cell_reference] = new_value
            
            # Getting the updated value of the cell
            updated_value = sheet[cell_reference].value
            
            # Printing the updated value after the update
            print(f"Updated value of '{name}' in {sheet_name} ({cell_reference}): {updated_value}")
        
        except KeyError:
            print(f"Error: Named range '{name}' not found in the workbook.")
        except Exception as e:
            print(f"An error occurred: {e}")

In [9]:
# Extracting project data from the template file
def extract_project_data_pandas(template_file_path):

    df = pd.read_excel(template_file_path, sheet_name="Overall Info")  
    df_filtered = df[df['ProjID'].notna()]  
    
    name_value_map = df_filtered.set_index('ProjID').to_dict(orient='index')
    
    return name_value_map


In [10]:
# Using the extracting project function 
name_value_map = extract_project_data_pandas(template_file_path)

In [11]:
#Checking generated dictionary 
print(name_value_map)

{1.0: {'ProjName': 'Sketch Example', 'District': 'HQ', 'ProjType': '    Hwy-Rail Grade Crossing', 'ProjLoc': 1.0, 'Construct': 6.0, 'NumDirections': 2.0, 'PeakLngthNB': 5.0, 'RoadTypeNB': 'C', 'RoadTypeB': 'C', 'GenLanesNB': 2.0, 'GenLanesB': 2.0, 'HOVLanesNB': 0.0, 'HOVLanesB': 2.0, 'HOVRest': 2.0, 'Exclusive': 'N', 'FFSpeedNB': 65.0, 'FFSpeedB': 65.0, 'RampFFSpdNB': 35.0, 'RampFFSpdB': 35.0, 'SegmentNB': 1.1, 'SegmentB': 1.1, 'ImpactedNB': 1.1, 'ImpactedB': 1.1, 'ADT0': 90000.0, 'ADT1NB': 101617.2, 'ADT1B': 109265.8064516129, 'ADT20NB': 138405.0, 'ADT20B': 148822.58064516127, 'HOVvolNB': 0.0, 'HOVvolB': 2030.0, 'PerIndHOV': '100%', 'PerWeaveNB': nan, 'PerWeaveB': '10%', 'PerTruckNB': 0.09, 'PerTruckB': 0.09, 'TruckSpeed': 55.0, 'RampVolP': 0.0, 'RampVolNP': 0.0, 'MeterStrat': 0.0, 'ArrRate1': 8468.1, 'ArrRate20': 11533.75, 'DepRate1': 2800.0, 'DepRate20': 2800.0, 'IRI1NB': 0.0, 'IRI1B': 0.0, 'IRI20NB': nan, 'IRI20B': nan, 'AVONonNB': 1.38, 'AVONonB': 1.65, 'AVOPeakNB': 1.42, 'AVOPeak

In [12]:
def recalculate_and_save_xlsm(wb, excel_file_path, name_value_map, output_file_path_prefix):
    # Iterating through the name_value_map, where each key is a project ID and the value is its corresponding data
    for key, value in name_value_map.items():
        update_named_ranges(wb, value) # Update named ranges in the workbook using the current project's data
    
        temp_local_path = os.path.join(os.path.dirname(excel_file_path), "temp_recalculated.xlsm") #path for a temporary file that hold the recalculated workbook 
        wb.save(temp_local_path)
    
        try:
            with xw.App(visible=False) as app: #opens excel file in the background 
                print("Starting Excel application...")

                #opening the recalculated workbook
                wb_excel = app.books.open(temp_local_path)
                print(f"Workbook {temp_local_path} opened successfully.")
            

                wb_excel.app.calculate()  #excel recalculates all the formulas 
                print("Excel formulas recalculated.")
            
                output_file_path = f"{output_file_path_prefix}_{key}.xlsm" 
                wb_excel.save(output_file_path) #recalculated files saved in designated output file path 
                wb_excel.close()
            
                print(f"Workbook recalculated and saved to {output_file_path}.")
    
        except Exception as e:
            print(f"Error during recalculation: {e}") #print errors
    
        os.remove(temp_local_path)
    
        print("Temporary file removed.")

In [13]:
# Using the recalculate and save function 
recalculate_and_save_xlsm(wb, excel_file_path, name_value_map, output_dir)

Current value of 'ProjName' in 1) Project Information ($E$4): Hypothetical Project
Updated value of 'ProjName' in 1) Project Information ($E$4): Sketch Example
Error: Named range 'District' not found in the workbook.
Current value of 'ProjType' in 1) Project Information ($F$10): None
Updated value of 'ProjType' in 1) Project Information ($F$10):     Hwy-Rail Grade Crossing
Current value of 'ProjLoc' in 1) Project Information ($H$12): None
Updated value of 'ProjLoc' in 1) Project Information ($H$12): 1.0
Current value of 'Construct' in 1) Project Information ($F$14): None
Updated value of 'Construct' in 1) Project Information ($F$14): 6.0
Current value of 'NumDirections' in 1) Project Information ($F$15): 2
Updated value of 'NumDirections' in 1) Project Information ($F$15): 2.0
Current value of 'PeakLngthNB' in 1) Project Information ($F$17): 5
Updated value of 'PeakLngthNB' in 1) Project Information ($F$17): 5.0
Current value of 'RoadTypeNB' in 1) Project Information ($G$24): =IF(HwyRa